- Створення програми, яка буде читати дані з одного Kafka-топіку за допомогою Spark Streaming, робити просте перетворення й записувати в інший топік.

- Валідація обробки даних з попереднього пункту за допомогою додаткового Consumer.

In [1]:
import os

os.environ["PYTHONUNBUFFERED"] = "1"
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17"
os.environ["SPARK_HOME"] = "/opt/homebrew/Cellar/apache-spark/4.0.0/libexec"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["SPARK_HOME"] + "/bin:" + os.environ["PATH"]

In [2]:
os.environ["PATH"]

'/opt/homebrew/opt/openjdk@17/bin:/opt/homebrew/Cellar/apache-spark/4.0.0/libexec/bin:/Users/aleksejkitajskij/Desktop/goit-de-hw-06/.venv/bin:/opt/homebrew/opt/apache-spark/libexec/bin:/opt/anaconda3/bin:/opt/anaconda3/condabin:/Users/aleksejkitajskij/fam_terminal:/opt/homebrew/opt/openjdk@17/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/Library/Frameworks/Python.framework/Versions/3.12/bin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/Library/Frameworks/Python.framework/Versions/3.11/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/usr/local/share/dotnet:~/.dotnet/tools'

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType
from pyspark.sql import SparkSession
from configs import kafka_config

In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = ('--packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.1 pyspark-shell')

In [5]:
# Створення SparkSession
spark = (SparkSession.builder
         .appName("KafkaStreaming")
         .master("local[*]")
         .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/26 10:03:52 WARN Utils: Your hostname, Aleksejs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.226 instead (on interface en0)
25/08/26 10:03:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/4.0.0/libexec/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/aleksejkitajskij/.ivy2.5.2/cache
The jars for the packages stored in: /Users/aleksejkitajskij/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f7b8e94-8f47-46d1-86ca-087c1e6f2f18;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.

In [6]:
# Читання потоку даних із Kafka
# Вказівки, як саме ми будемо під'єднуватися, паролі, протоколи
# maxOffsetsPerTrigger - будемо читати 5 записів за 1 тригер.
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_config['bootstrap_servers'][0]) \
    .option("kafka.security.protocol", "SASL_PLAINTEXT") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config",
            'org.apache.kafka.common.security.plain.PlainLoginModule required username="admin" password="VawEzo1ikLtrA8Ug8THa";') \
    .option("subscribe", "barka__streaming_in") \
    .option("startingOffsets", "earliest") \
    .option("maxOffsetsPerTrigger", "5") \
    .load()

In [7]:
json_schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("temperature", IntegerType(), True),
    StructField("humidity", IntegerType(), True)
])

In [8]:
parsed = df.selectExpr("CAST(key AS STRING) AS key", "CAST(value AS STRING) AS value_str") \
    .withColumn("value_json", from_json(col("value_str"), json_schema)) \
    .withColumn("timestamp", to_timestamp(col("value_json.timestamp"))) \
    .withColumn("temperature", col("value_json.temperature")) \
    .withColumn("humidity", col("value_json.humidity")) \
    .drop("value_str", "value_json")

In [9]:
# Виведення даних на екран
displaying_df = parsed.writeStream \
    .trigger(availableNow=True) \
    .outputMode("append") \
    .format("console") \
    .option("checkpointLocation", "/tmp/checkpoints-98") \
    .start() \
    .awaitTermination()

25/08/26 10:03:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/08/26 10:03:56 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+--------------------+-----------+--------+
|                 key|           timestamp|temperature|humidity|
+--------------------+--------------------+-----------+--------+
|"ff9f8748-a1df-4c...|2025-08-26 10:02:...|         41|      18|
|"ff9f8748-a1df-4c...|2025-08-26 10:02:...|         37|      57|
|"ff9f8748-a1df-4c...|2025-08-26 10:02:...|         39|      78|
|"ff9f8748-a1df-4c...|2025-08-26 10:02:...|         41|      29|
|"ff9f8748-a1df-4c...|2025-08-26 10:02:...|         41|      31|
+--------------------+--------------------+-----------+--------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+--------------------+-----------+--------+
|                 key|           timestamp|temperature|humidity|
+--------------------+--------------------+-----------+--------+
|"ff9f8748-a1df-4c...|202

In [ ]:

# Виведення даних на екран
displaying_df = clean_df.writeStream \
    .trigger(availableNow=True) \
    .outputMode("append") \
    .format("console") \
    .option("checkpointLocation", "/tmp/checkpoints-2") \
    .start() \
    .awaitTermination()

# Підготовка даних для запису в Kafka: формування ключ-значення
prepare_to_kafka_df = clean_df.select(
    col("key"),
    to_json(struct(col("value"), col("new_value"))).alias("value")
)

# Запис оброблених даних у Kafka-топік 'oleksiy_spark_streaming_out'
query = prepare_to_kafka_df.writeStream \
    .trigger(processingTime='5 seconds') \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "77.81.230.104:9092") \
    .option("topic", "oleksiy_spark_streaming_out") \
    .option("kafka.security.protocol", "SASL_PLAINTEXT") \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.sasl.jaas.config",
            "org.apache.kafka.common.security.plain.PlainLoginModule required username='admin' password='VawEzo1ikLtrA8Ug8THa';") \
    .option("checkpointLocation", "/tmp/checkpoints-3") \
    .start() \
    .awaitTermination()